In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

class DataHandler:
    def __init__(self, symbol: str, start: str, end: str):
        self.symbol = symbol
        self.start = start
        self.end = end
    
    def load_data(self) -> pd.DataFrame:
        data = yf.download(self.symbol, start=self.start, end=self.end, progress=False, auto_adjust=False)
        if data.empty:
            raise ValueError(f"No data fetched for {self.symbol}. Check symbol and dates.")
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        data = data.dropna()
        print(f"Loaded {len(data)} rows of data for {self.symbol} from {data.index[0]} to {data.index[-1]}")
        return data

class SMACrossoverStrategy:
    def __init__(self, short_window: int = 50, long_window: int = 200):
        if short_window >= long_window:
            raise ValueError("Short window must be less than long window.")
        self.short_window = short_window
        self.long_window = long_window
    
    def generate_signals(self, data: pd.DataFrame) -> pd.DataFrame:
        data = data.copy()
        price = data['Close']
        if isinstance(price, pd.DataFrame):
            price = price.squeeze()
        data['short_ma'] = price.rolling(window=self.short_window).mean()
        data['long_ma']  = price.rolling(window=self.long_window).mean()
        data['signal'] = 0
        valid_start = self.long_window
        data.loc[data.index[valid_start:], 'signal'] = np.where(
            data['short_ma'][valid_start:] > data['long_ma'][valid_start:], 1, -1
        )
        data['positions'] = data['signal'].diff().fillna(0)
        num_trades = int(data['positions'].abs().sum())
        print(f"Generated {num_trades} trades (buy/sell signals)")
        if num_trades == 0:
            print("Warning: No trades generated. Check data or parameters.")
        return data


class Backtester:
    def __init__(self, initial_capital: float = 100000, commission: float = 0.002, slippage: float = 0.001):
        self.initial_capital = initial_capital
        self.commission = commission
        self.slippage = slippage
    
    def run(self, data: pd.DataFrame) -> tuple:
        portfolio = pd.DataFrame(index=data.index)
        portfolio['positions'] = data['signal'] * 100

        pos_diff = portfolio['positions'].diff().fillna(0)
        slippage_factor = 1 + (self.slippage * np.sign(pos_diff))

        close = data['Close']
        if isinstance(close, pd.DataFrame):
            close = close.squeeze()

        effective_prices = close * slippage_factor
        portfolio['holdings'] = portfolio['positions'] * close

        trade_costs = np.abs(pos_diff) * effective_prices
        commission_costs = trade_costs * self.commission
        total_costs = (trade_costs + commission_costs).fillna(0)

        portfolio['cash'] = self.initial_capital - total_costs.cumsum()
        portfolio['total'] = portfolio['cash'] + portfolio['holdings']
        portfolio['returns'] = portfolio['total'].pct_change().fillna(0)
        print(f"Backtest complete. Final portfolio value: ${portfolio['total'].iloc[-1]:,.2f}")
        return portfolio, trade_costs, commission_costs


class PerformanceMetrics:
    @staticmethod
    def calculate_metrics(portfolio: pd.DataFrame, data: pd.DataFrame) -> dict:
        returns = portfolio['returns']
        annual_returns = returns.mean() * 252
        annual_vol = returns.std() * np.sqrt(252)
        risk_free_rate = 0.01
        sharpe_ratio = (annual_returns - risk_free_rate) / annual_vol if annual_vol > 0 else np.nan
        rolling_max = portfolio['total'].cummax()
        drawdown = (portfolio['total'] - rolling_max) / rolling_max
        max_drawdown = drawdown.min()
        holding_returns = returns[portfolio['positions'].abs() > 0]
        win_rate = (holding_returns > 0).mean() if len(holding_returns) > 0 else 0
        return {
            'Annual Return': annual_returns,
            'Annual Volatility': annual_vol,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown': max_drawdown,
            'Win Rate': win_rate
        }

def plot_results(data: pd.DataFrame, portfolio: pd.DataFrame, save_fig: bool = False):
    global strategy, backtester, data_handler
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    ax1.plot(data.index, data['Close'], label='Close Price', alpha=0.7, linewidth=1)
    ax1.plot(data.index, data['short_ma'], label=f'Short MA ({strategy.short_window}d)', alpha=0.8)
    ax1.plot(data.index, data['long_ma'], label=f'Long MA ({strategy.long_window}d)', alpha=0.8)
    buy_signals = data[data['positions'] > 0]
    sell_signals = data[data['positions'] < 0]
    if not buy_signals.empty:
        ax1.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', s=50, label='Buy Signal', zorder=5)
    if not sell_signals.empty:
        ax1.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', s=50, label='Sell Signal', zorder=5)
    ax1.set_title(f'{data_handler.symbol} Price and Trading Signals')
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax2.plot(portfolio.index, portfolio['total'], label='Portfolio Value', color='blue', linewidth=2)
    ax2.axhline(y=backtester.initial_capital, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
    ax2.set_title('Portfolio Equity Curve')
    ax2.set_ylabel('Portfolio Value ($)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    rolling_max = portfolio['total'].cummax()
    drawdown = (portfolio['total'] - rolling_max) / rolling_max * 100
    ax3.fill_between(portfolio.index, drawdown, 0, color='red', alpha=0.3)
    ax3.plot(portfolio.index, drawdown, color='red', linewidth=1)
    ax3.set_title('Portfolio Drawdown')
    ax3.set_ylabel('Drawdown (%)')
    ax3.set_xlabel('Date')
    ax3.grid(True, alpha=0.3)
    plt.tight_layout()
    if save_fig:
        plt.savefig('backtest_results.png', dpi=300, bbox_inches='tight')
        print("Plot saved as 'backtest_results.png'")
    plt.show()

symbol = 'AAPL'
start_date = '2018-01-01'
end_date = '2023-12-31'

try:
    data_handler = DataHandler(symbol=symbol, start=start_date, end=end_date)
    strategy = SMACrossoverStrategy(short_window=50, long_window=200)
    backtester = Backtester(initial_capital=100000, commission=0.002, slippage=0.001)
    data = data_handler.load_data()
    data = strategy.generate_signals(data)
    portfolio, trade_costs, commission_costs = backtester.run(data)
    metrics = PerformanceMetrics.calculate_metrics(portfolio, data)
    print("\n=== Performance Metrics ===")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")
    plot_results(data, portfolio, save_fig=False)
    print("\n=== Sample Portfolio Data (Last 5 Rows) ===")
    print(portfolio.tail())
except Exception as e:
    print(f"Error during execution: {str(e)}")